In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

import os
os.listdir("/content/drive/MyDrive/AutoModelForSequenceClassification/")


In [ ]:
!pip install transformers datasets
!pip install torch

In [ ]:
import pandas as pd
import re
from difflib import SequenceMatcher

# ===============================
# تحميل ملف CSV
# ===============================
csv_path = "/content/drive/MyDrive/AutoModelForSequenceClassification/updates.csv"

try:
    df = pd.read_csv(csv_path, encoding='utf-8')
    print("✅ تم تحميل الملف بنجاح")
except UnicodeDecodeError:
    try:
        df = pd.read_csv(csv_path, encoding='utf-8-sig')
        print("✅ تم تحميل الملف بنجاح (utf-8-sig)")
    except:
        df = pd.read_csv(csv_path, encoding='cp1256')
        print("✅ تم تحميل الملف بنجاح (cp1256)")
except Exception as e:
    print(f"❌ خطأ في تحميل الملف: {str(e)}")
    raise

print(f"\n✅ عدد المنتجات: {len(df)}")
print(f"✅ عدد الأعمدة: {len(df.columns)}")

# ===============================
# تنظيف النصوص العربية
# ===============================
def normalize_arabic(text):
    """تنظيف وتوحيد النصوص العربية"""
    if pd.isna(text):
        return ""

    text = str(text).strip()
    text = re.sub(r'[إأآا]', 'ا', text)
    text = re.sub(r'ة', 'ه', text)
    text = re.sub(r'ى', 'ي', text)
    text = re.sub(r'ئ', 'ء', text)
    text = re.sub(r'[^\w\s]', ' ', text)
    text = ' '.join(text.split())

    return text.lower()

# ===============================
# البحث بالجملة الكاملة (مش كل كلمة لوحدها!)
# ===============================
def phrase_search_score(search_phrase, target_text):
    """
    البحث بالجملة الكاملة - الجملة لازم تكون موجودة كلها مع بعض

    مثال:
    - search_phrase = "حليب كامل الدسم"
    - target_text = "حليب كامل الدسم UHT معقم"
    - النتيجة: ✅ موجودة (الجملة كلها موجودة)

    - target_text = "حليب نصف دسم"
    - النتيجة: ❌ مش موجودة (مش نفس الجملة)
    """
    if not search_phrase or not target_text:
        return 0.0

    search_clean = normalize_arabic(search_phrase)
    target_clean = normalize_arabic(target_text)

    # 1. الجملة موجودة بالضبط
    if search_clean == target_clean:
        return 1.0

    # 2. الجملة موجودة كجزء من النص
    if search_clean in target_clean:
        # نحسب كام نسبة الجملة من النص الكلي
        phrase_length = len(search_clean)
        target_length = len(target_clean)
        coverage = phrase_length / target_length

        # كل ما الجملة تكون جزء أكبر من النص، الدرجة أعلى
        base_score = 0.85
        bonus = coverage * 0.15
        return min(base_score + bonus, 1.0)

    # 3. التحقق من التشابه العالي للجملة
    similarity = SequenceMatcher(None, search_clean, target_clean).ratio()
    if similarity >= 0.9:
        return similarity

    # 4. التحقق من وجود الجملة بترتيب الكلمات (حتى لو فيه كلمات زيادة بينهم)
    search_words = search_clean.split()
    target_words = target_clean.split()

    # البحث عن تسلسل الكلمات
    if len(search_words) > 1:
        # نشوف لو كل الكلمات موجودة بالترتيب
        search_index = 0
        for target_word in target_words:
            if search_index < len(search_words):
                if search_words[search_index] in target_word or target_word in search_words[search_index]:
                    search_index += 1

        # لو لقينا كل الكلمات بالترتيب
        if search_index == len(search_words):
            # نحسب درجة حسب المسافة بين الكلمات
            match_ratio = len(search_words) / len(target_words)
            return 0.7 + (match_ratio * 0.2)

    # 5. لو الجملة مش موجودة خالص
    return 0.0

# ===============================
# البحث المحسّن
# ===============================
def enhanced_search(product_name="", sector_name="", min_confidence=0.7, top_n=30):
    """
    بحث محسّن - يبحث عن الجملة كاملة مش كل كلمة لوحدها

    Parameters:
    - product_name: اسم المنتج (جملة كاملة)
    - sector_name: اسم القطاع (جملة كاملة)
    - min_confidence: الحد الأدنى للثقة (افتراضي 70%)
    - top_n: عدد النتائج المطلوبة
    """

    if not product_name and not sector_name:
        return {
            "found": False,
            "message": "⚠️ يجب إدخال اسم المنتج أو القطاع على الأقل"
        }

    results = []

    # البحث في كل صف
    for index, row in df.iterrows():
        row_product = str(row.get('اسم المنتج', row.get('product_name', row.get('المنتج', ''))))
        row_sector = str(row.get('القطاع', row.get('sector', row.get('Sector', ''))))

        if row_product == "nan":
            row_product = ""
        if row_sector == "nan":
            row_sector = ""

        # حساب درجة تطابق المنتج (بالجملة الكاملة)
        product_score = 0.0
        if product_name:
            product_score = phrase_search_score(product_name, row_product)
        else:
            product_score = 0.5

        # حساب درجة تطابق القطاع (بالجملة الكاملة)
        sector_score = 0.0
        if sector_name:
            sector_score = phrase_search_score(sector_name, row_sector)
        else:
            sector_score = 0.5

        # حساب الثقة الإجمالية (وزن أكبر للمنتج)
        total_confidence = (product_score * 0.7) + (sector_score * 0.3)

        # فلترة حسب الحد الأدنى للثقة
        if total_confidence < min_confidence:
            continue

        # جمع التصنيفات
        classifications = {}
        for col in df.columns:
            if col not in ['اسم المنتج', 'القطاع', 'product_name', 'sector', 'Sector', 'المنتج']:
                value = str(row.get(col, ""))
                if value and value != "nan" and value.strip():
                    classifications[col] = value

        # تحديد نوع التطابق
        match_details = []
        if product_name and product_score > 0.5:
            if product_score >= 0.95:
                match_details.append("✓ مطابق تماماً")
            elif product_score >= 0.85:
                match_details.append("✓ يحتوي على الجملة كاملة")
            elif product_score >= 0.7:
                match_details.append("تطابق قوي للجملة")
            else:
                match_details.append("تطابق جزئي")

        if sector_name and sector_score > 0.5:
            if sector_score >= 0.95:
                match_details.append("✓ قطاع مطابق تماماً")
            elif sector_score >= 0.85:
                match_details.append("✓ يحتوي على القطاع")
            elif sector_score >= 0.7:
                match_details.append("قطاع مطابق")

        results.append({
            "اسم_المنتج": row_product if row_product else "غير محدد",
            "القطاع": row_sector if row_sector else "غير محدد",
            "التصنيفات": classifications,
            "الثقة": total_confidence,
            "درجة_المنتج": product_score,
            "درجة_القطاع": sector_score,
            "نوع_المطابقة": " | ".join(match_details) if match_details else "عام",
            "index": index
        })

    # ترتيب النتائج حسب الثقة
    results.sort(key=lambda x: x["الثقة"], reverse=True)

    if results:
        search_info = []
        if product_name:
            search_info.append(f"📦 المنتج: '{product_name}'")
        if sector_name:
            search_info.append(f"🏢 القطاع: '{sector_name}'")

        return {
            "found": True,
            "query": " | ".join(search_info),
            "count": len(results),
            "results": results[:top_n]
        }
    else:
        return {
            "found": False,
            "query": f"{product_name} - {sector_name}",
            "message": f"❌ لم يتم العثور على المنتج/القطاع بالجملة المحددة (ثقة أقل من {min_confidence:.0%})"
        }

# ===============================
# عرض النتائج
# ===============================
def display_results(result, show_all_classifications=True, show_scores=True):
    """عرض نتائج البحث بشكل منظم"""

    print("\n" + "=" * 100)
    print(f"🔍 نتائج البحث: {result.get('query', 'N/A')}")
    print("=" * 100)

    if not result["found"]:
        print(f"\n{result.get('message', 'لم يتم العثور على نتائج')}\n")
        return

    print(f"\n✅ تم العثور على {result['count']} نتيجة:\n")

    for i, item in enumerate(result["results"], 1):
        confidence = item['الثقة']

        # تحديد أيقونة الثقة
        if confidence >= 0.9:
            icon = "🟢"
            level = "ممتاز"
        elif confidence >= 0.75:
            icon = "🟡"
            level = "جيد"
        elif confidence >= 0.6:
            icon = "🟠"
            level = "متوسط"
        else:
            icon = "🔴"
            level = "ضعيف"

        print(f"{i:3d}. {icon} [{level}] الثقة: {confidence:.3f}")
        print(f"     📦 المنتج: {item['اسم_المنتج']}")
        print(f"     🏢 القطاع: {item['القطاع']}")
        print(f"     🎯 {item['نوع_المطابقة']}")

        if show_scores:
            print(f"     📊 درجة المنتج: {item['درجة_المنتج']:.3f} | درجة القطاع: {item['درجة_القطاع']:.3f}")

        if item['التصنيفات'] and show_all_classifications:
            print(f"     📋 التصنيفات:")
            for class_name, class_value in list(item['التصنيفات'].items())[:5]:
                short_name = class_name[:40] + "..." if len(class_name) > 40 else class_name
                short_value = class_value[:55] + "..." if len(class_value) > 55 else class_value
                print(f"        • {short_name}: {short_value}")

            if len(item['التصنيفات']) > 5:
                print(f"        ... و {len(item['التصنيفات']) - 5} تصنيف آخر")

        print()

        if i % 5 == 0 and i < len(result["results"]):
            print("   " + "-" * 90)

    print("=" * 100 + "\n")

def quick_display(result):
    """عرض مختصر للنتائج"""

    print(f"\n🔍 {result.get('query', 'بحث')}")
    print("─" * 80)

    if not result["found"]:
        print(f"❌ {result.get('message', 'لا توجد نتائج')}\n")
        return

    print(f"✅ {result['count']} نتيجة\n")

    for i, item in enumerate(result["results"][:15], 1):
        if item['الثقة'] >= 0.9:
            conf_icon = "🟢"
        elif item['الثقة'] >= 0.75:
            conf_icon = "🟡"
        else:
            conf_icon = "🟠"

        print(f"{i:2d}. {conf_icon} {item['اسم_المنتج'][:65]}")
        print(f"    القطاع: {item['القطاع'][:40]} | ثقة: {item['الثقة']:.3f}")
        print(f"    {item['نوع_المطابقة']}")
        print()

    if result['count'] > 15:
        print(f"... و {result['count'] - 15} نتيجة أخرى\n")

# ===============================
# دالة بحث سريع ⚡
# ===============================
def s(product="", sector="", conf=0.7, n=20, show_full=False):
    """
    بحث سريع بالجملة الكاملة

    الاستخدام:
    s("حليب كامل الدسم")              # بحث بالجملة كاملة
    s("أسمنت بورتلاند", "مواد بناء")  # منتج + قطاع
    s(sector="منتجات ألبان")          # قطاع فقط
    s("حليب كامل الدسم", conf=0.85)  # رفع الحد الأدنى للثقة
    s("حليب", n=30)                   # 30 نتيجة
    s("حليب", show_full=True)        # عرض كامل بالتفاصيل

    ملاحظة: البحث يدور على الجملة كلها مع بعض، مش كل كلمة لوحدها!
    """
    result = enhanced_search(
        product_name=product,
        sector_name=sector,
        min_confidence=conf,
        top_n=n
    )

    if show_full:
        display_results(result, show_all_classifications=True, show_scores=True)
    else:
        quick_display(result)

    return result

# ===============================
# أمثلة للاستخدام
# ===============================
def test_search():
    """اختبارات البحث بالجملة الكاملة"""

    print("\n" + "🧪" * 40)
    print("اختبار البحث بالجملة الكاملة")
    print("🧪" * 40 + "\n")

    # مثال 1: بحث بجملة كاملة
    print("1️⃣ البحث عن: 'حليب كامل الدسم' (جملة كاملة)")
    result1 = enhanced_search(product_name="حليب كامل الدسم")
    quick_display(result1)

    # مثال 2: بحث في القطاع
    print("\n" + "="*80)
    print("2️⃣ البحث عن قطاع: 'منتجات ألبان'")
    result2 = enhanced_search(sector_name="منتجات ألبان")
    quick_display(result2)

    # مثال 3: بحث مركب
    print("\n" + "="*80)
    print("3️⃣ بحث مركب: 'أسمنت بورتلاند' في 'مواد بناء'")
    result3 = enhanced_search(
        product_name="أسمنت بورتلاند",
        sector_name="مواد بناء",
        min_confidence=0.7
    )
    display_results(result3, show_all_classifications=True, show_scores=True)

# ===============================
# التشغيل
# ===============================
if __name__ == "__main__":
    print("\n📊 معلومات الملف:")
    print(f"   • عدد الصفوف: {len(df)}")
    print(f"   • عدد الأعمدة: {len(df.columns)}")

    print("\n💡 ملاحظة مهمة:")
    print("   البحث الآن يدور على الجملة كاملة، مش كل كلمة لوحدها!")
    print("   مثال: 'حليب كامل الدسم' ← يدور على الجملة دي كلها\n")

    # تشغيل الاختبارات
    test_search()

    print("\n" + "💡" * 40)
    print("✅ للاستخدام السريع:")
    print("   s('حليب كامل الدسم')       # بحث بالجملة")
    print("   s('أسمنت', 'بناء')         # منتج + قطاع")
    print("   s('حليب', conf=0.85)       # رفع الثقة")
    print("💡" * 40)

In [ ]:
pip install flask pandas fuzzywuzzy

In [ ]:
import pandas as pd
import re
from difflib import SequenceMatcher
from flask import Flask, request, jsonify

# ===============================
# تهيئة التطبيق وتحميل البيانات
# ===============================

# إنشاء تطبيق Flask
app = Flask(__name__)

# المسار إلى ملف CSV (يرجى تعديله حسب بيئة التشغيل)
# ملاحظة: يجب أن يكون الملف متاحاً لتطبيق الـ API عند التشغيل
csv_path = "/content/drive/MyDrive/AutoModelForSequenceClassification/updates.csv"

try:
    df = pd.read_csv(csv_path, encoding='utf-8')
    print("✅ تم تحميل الملف بنجاح")
except UnicodeDecodeError:
    try:
        df = pd.read_csv(csv_path, encoding='utf-8-sig')
        print("✅ تم تحميل الملف بنجاح (utf-8-sig)")
    except:
        try:
            df = pd.read_csv(csv_path, encoding='cp1256')
            print("✅ تم تحميل الملف بنجاح (cp1256)")
        except Exception as e:
            print(f"❌ خطأ في تحميل الملف: {str(e)}")
            # إيقاف التطبيق إذا فشل التحميل لعدم وجود بيانات للبحث فيها
            raise RuntimeError("فشل تحميل ملف البيانات") from e

print(f"\n✅ عدد المنتجات: {len(df)}")
print(f"✅ عدد الأعمدة: {len(df.columns)}")

# ===============================
# دالة تنظيف النصوص العربية (مُنسوخة من كودك)
# ===============================
def normalize_arabic(text):
    """تنظيف وتوحيد النصوص العربية"""
    if pd.isna(text):
        return ""

    text = str(text).strip()
    text = re.sub(r'[إأآا]', 'ا', text)
    text = re.sub(r'ة', 'ه', text)
    text = re.sub(r'ى', 'ي', text)
    text = re.sub(r'ئ', 'ء', text)
    text = re.sub(r'[^\w\s]', ' ', text)
    text = ' '.join(text.split())

    return text.lower()

# ===============================
# دالة حساب درجة تطابق الجملة (مُنسوخة من كودك)
# ===============================
def phrase_search_score(search_phrase, target_text):
    """
    البحث بالجملة الكاملة - الجملة لازم تكون موجودة كلها مع بعض
    ... (باقي الكود كما هو)
    """
    if not search_phrase or not target_text:
        return 0.0

    search_clean = normalize_arabic(search_phrase)
    target_clean = normalize_arabic(target_text)

    # 1. الجملة موجودة بالضبط
    if search_clean == target_clean:
        return 1.0

    # 2. الجملة موجودة كجزء من النص
    if search_clean in target_clean:
        phrase_length = len(search_clean)
        target_length = len(target_clean)
        coverage = phrase_length / target_length
        base_score = 0.85
        bonus = coverage * 0.15
        return min(base_score + bonus, 1.0)

    # 3. التحقق من التشابه العالي للجملة
    similarity = SequenceMatcher(None, search_clean, target_clean).ratio()
    if similarity >= 0.9:
        return similarity

    # 4. التحقق من وجود الجملة بترتيب الكلمات
    search_words = search_clean.split()
    target_words = target_clean.split()

    if len(search_words) > 1:
        search_index = 0
        for target_word in target_words:
            if search_index < len(search_words):
                if search_words[search_index] in target_word or target_word in search_words[search_index]:
                    search_index += 1

        if search_index == len(search_words):
            match_ratio = len(search_words) / len(target_words)
            return 0.7 + (match_ratio * 0.2)

    # 5. لو الجملة مش موجودة خالص
    return 0.0

# ===============================
# دالة البحث المحسّن (مُنسوخة من كودك)
# ===============================
def enhanced_search(product_name="", sector_name="", min_confidence=0.7, top_n=30):
    """
    بحث محسّن - يبحث عن الجملة كاملة مش كل كلمة لوحدها
    ... (باقي الكود كما هو)
    """
    global df # للتأكد من استخدام DataFrame الذي تم تحميله في التهيئة

    if not product_name and not sector_name:
        return {
            "found": False,
            "message": "⚠️ يجب إدخال اسم المنتج أو القطاع على الأقل"
        }

    results = []

    # البحث في كل صف
    for index, row in df.iterrows():
        # ... (منطق استخراج اسم المنتج والقطاع والثقة الإجمالية كما هو في كودك)
        row_product = str(row.get('اسم المنتج', row.get('product_name', row.get('المنتج', ''))))
        row_sector = str(row.get('القطاع', row.get('sector', row.get('Sector', ''))))

        if row_product == "nan":
            row_product = ""
        if row_sector == "nan":
            row_sector = ""

        # حساب درجة تطابق المنتج
        product_score = 0.0
        if product_name:
            product_score = phrase_search_score(product_name, row_product)
        else:
            product_score = 0.5

        # حساب درجة تطابق القطاع
        sector_score = 0.0
        if sector_name:
            sector_score = phrase_search_score(sector_name, row_sector)
        else:
            sector_score = 0.5

        # حساب الثقة الإجمالية (وزن أكبر للمنتج)
        total_confidence = (product_score * 0.7) + (sector_score * 0.3)

        # فلترة حسب الحد الأدنى للثقة
        if total_confidence < min_confidence:
            continue

        # جمع التصنيفات
        classifications = {}
        for col in df.columns:
            if col not in ['اسم المنتج', 'القطاع', 'product_name', 'sector', 'Sector', 'المنتج']:
                value = str(row.get(col, ""))
                if value and value != "nan" and value.strip():
                    classifications[col] = value

        # تحديد نوع التطابق
        match_details = []
        if product_name and product_score > 0.5:
            if product_score >= 0.95:
                match_details.append("✓ مطابق تماماً")
            elif product_score >= 0.85:
                match_details.append("✓ يحتوي على الجملة كاملة")
            elif product_score >= 0.7:
                match_details.append("تطابق قوي للجملة")
            else:
                match_details.append("تطابق جزئي")

        if sector_name and sector_score > 0.5:
            if sector_score >= 0.95:
                match_details.append("✓ قطاع مطابق تماماً")
            elif sector_score >= 0.85:
                match_details.append("✓ يحتوي على القطاع")
            elif sector_score >= 0.7:
                match_details.append("قطاع مطابق")


        results.append({
            "اسم_المنتج": row_product if row_product else "غير محدد",
            "القطاع": row_sector if row_sector else "غير محدد",
            "التصنيفات": classifications,
            "الثقة": round(total_confidence, 4), # تقريب لأربع منازل
            "درجة_المنتج": round(product_score, 4),
            "درجة_القطاع": round(sector_score, 4),
            "نوع_المطابقة": " | ".join(match_details) if match_details else "عام",
            "index": int(index) # التأكد من أن index هو عدد صحيح قابل للتسلسل في JSON
        })

    # ترتيب النتائج حسب الثقة
    results.sort(key=lambda x: x["الثقة"], reverse=True)

    if results:
        search_info = []
        if product_name:
            search_info.append(f"📦 المنتج: '{product_name}'")
        if sector_name:
            search_info.append(f"🏢 القطاع: '{sector_name}'")

        return {
            "found": True,
            "query": " | ".join(search_info),
            "count": len(results),
            "results": results[:top_n]
        }
    else:
        return {
            "found": False,
            "query": f"{product_name} - {sector_name}",
            "message": f"❌ لم يتم العثور على المنتج/القطاع بالجملة المحددة (ثقة أقل من {min_confidence:.0%})"
        }


# ===============================
# تعريف مسار الـ API (API Endpoint)
# ===============================
@app.route('/search', methods=['GET'])
def search_api():
    """
    واجهة API للبحث عن المنتج والقطاع.
    المعاملات:
    - product_name (اختياري)
    - sector_name (اختياري)
    - min_confidence (اختياري، القيمة الافتراضية 0.7)
    - top_n (اختياري، القيمة الافتراضية 30)
    """

    # استخراج المعاملات من طلب GET
    product = request.args.get('product_name', default="", type=str)
    sector = request.args.get('sector_name', default="", type=str)

    # تحويل الثقة وعدد النتائج إلى أرقام
    try:
        min_conf = float(request.args.get('min_confidence', default=0.7))
        top_results = int(request.args.get('top_n', default=30))
    except ValueError:
        return jsonify({
            "error": "min_confidence و top_n يجب أن يكونا أرقاماً صحيحة/عشرية"
        }), 400

    # تنفيذ دالة البحث المحسّن
    search_result = enhanced_search(
        product_name=product,
        sector_name=sector,
        min_confidence=min_conf,
        top_n=top_results
    )

    # إرجاع النتائج بتنسيق JSON
    return jsonify(search_result)

# ===============================
# تشغيل التطبيق
# ===============================


In [ ]:
!pip install pyngrok --quiet

from pyngrok import ngrok
from threading import Thread
import getpass

# 1) Auth token (only once per runtime)
print("Enter your Ngrok Auth Token:")
ngrok.set_auth_token(getpass.getpass())

# 2) Function to run Flask in background
def run_flask():
    print("\n--- تشغيل خادم Flask ---")
    print("🚀 API جاهزة على: http://127.0.0.1:5000/search")
    # important: host=0.0.0.0 so ngrok can see it
    app.run(host="0.0.0.0", port=5000, debug=False, use_reloader=False)

# 3) Start Flask in a background thread (non-blocking)
flask_thread = Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

# 4) Open ngrok tunnel to port 5000
public_url = ngrok.connect(5000).public_url
print(f"\n✅ Ngrok Tunnel ready!")
print(f"🔗 Public API URL: {public_url}/search")


In [ ]:
# بحث بجملة كاملة
s(" كامل الدسم")